In [1]:
import configparser
import unittest
import datetime
import sys
import pandas as pd

# from tqdm import tqdm_notebook

sys.path.append("/home/sergey/drclinics/common")
from universal_connection import UniversalConnection, DBType

sys.path.append("/home/sergey/drclinics/reports")
from report_utils import excel_report

import numpy as np
from log import log

In [2]:
# проверка телефона на валидность

def valid_phone(phone):
    try:
        if len(phone) == 11:
            return phone
        else:
            return 'Invalid phone'
    except:
        return 'Invalid phone'

In [3]:
# получение всех консультаций

sql = """
with temp as (
select a.id appoint_id,
	   p.person_id,
	   per.phone,
	   --per.partner_id,
	   --par."name", 
	   case 
        	when promo.description_tsvector = 'телемедицина_вэб' then 'VEBMED'
        	else 'ПРОЧИЕ' 
       end "name",
	   case
	   		when (((substring(per.phone, 2, 3) = '901' and cast(substring(per.phone, 5, 7) as numeric) between 9460000 and 9469999) 
	   			 or (substring(per.phone, 2, 3) = '902' and cast(substring(per.phone, 5, 7) as numeric) between 7110000 and 7119999)
	   			 or (substring(per.phone, 2, 3) = '902' and cast(substring(per.phone, 5, 7) as numeric) between 7150000 and 7159999)
	   			 or (substring(per.phone, 2, 3) = '902' and cast(substring(per.phone, 5, 7) as numeric) between 7180000 and 7199999)
	   			 or (substring(per.phone, 2, 3) = '903' and cast(substring(per.phone, 5, 7) as numeric) between 0610000 and 0629999)
	   			 or (substring(per.phone, 2, 3) = '903' and cast(substring(per.phone, 5, 7) as numeric) between 3050000 and 3079999)
	   			 or (substring(per.phone, 2, 3) = '903' and cast(substring(per.phone, 5, 7) as numeric) between 3130000 and 3149999)
	   			 or (substring(per.phone, 2, 3) = '903' and cast(substring(per.phone, 5, 7) as numeric) between 3180000 and 3199999)
	   			 or (substring(per.phone, 2, 3) = '903' and cast(substring(per.phone, 5, 7) as numeric) between 3400000 and 3449999)
	   			 or (substring(per.phone, 2, 3) = '903' and cast(substring(per.phone, 5, 7) as numeric) between 3870000 and 3889999)
	   			 or (substring(per.phone, 2, 3) = '904' and cast(substring(per.phone, 5, 7) as numeric) between 6600000 and 6699999)
	   			 or (substring(per.phone, 2, 3) = '904' and cast(substring(per.phone, 5, 7) as numeric) between 6700000 and 6799999)
	   			 or (substring(per.phone, 2, 3) = '904' and cast(substring(per.phone, 5, 7) as numeric) between 7120000 and 7179999)
	   			 or (substring(per.phone, 2, 3) = '904' and cast(substring(per.phone, 5, 7) as numeric) between 7180000 and 7199999)
	   			 or (substring(per.phone, 2, 3) = '904' and cast(substring(per.phone, 5, 7) as numeric) between 7600000 and 7699999)
	   			 or (substring(per.phone, 2, 3) = '905' and cast(substring(per.phone, 5, 7) as numeric) between 0200000 and 0269999)
	   			 or (substring(per.phone, 2, 3) = '905' and cast(substring(per.phone, 5, 7) as numeric) between 0380000 and 0399999)
	   			 or (substring(per.phone, 2, 3) = '905' and cast(substring(per.phone, 5, 7) as numeric) between 3100000 and 3199999)
	   			 or (substring(per.phone, 2, 3) = '905' and cast(substring(per.phone, 5, 7) as numeric) between 3700000 and 3799999)
	   			 or (substring(per.phone, 2, 3) = '906' and cast(substring(per.phone, 5, 7) as numeric) between 1100000 and 1249999)
	   			 or (substring(per.phone, 2, 3) = '906' and cast(substring(per.phone, 5, 7) as numeric) between 3200000 and 3339999)
	   			 or (substring(per.phone, 2, 3) = '908' and cast(substring(per.phone, 5, 7) as numeric) between 3300000 and 3399999)
	   			 or (substring(per.phone, 2, 3) = '908' and cast(substring(per.phone, 5, 7) as numeric) between 3400000 and 3439999)
	   			 or (substring(per.phone, 2, 3) = '908' and cast(substring(per.phone, 5, 7) as numeric) between 3460000 and 3499999)
	   			 or (substring(per.phone, 2, 3) = '909' and cast(substring(per.phone, 5, 7) as numeric) between 3060000 and 3149999)
	   			 or (substring(per.phone, 2, 3) = '917' and cast(substring(per.phone, 5, 7) as numeric) between 2200000 and 2999999)
	   			 or (substring(per.phone, 2, 3) = '917' and cast(substring(per.phone, 5, 7) as numeric) between 3900000 and 3999999)
	   			 or (substring(per.phone, 2, 3) = '917' and cast(substring(per.phone, 5, 7) as numeric) between 8500000 and 9399999)
	   			 or (substring(per.phone, 2, 3) = '919' and cast(substring(per.phone, 5, 7) as numeric) between 6200000 and 6499999)
	   			 or (substring(per.phone, 2, 3) = '919' and cast(substring(per.phone, 5, 7) as numeric) between 6800000 and 6999999)
	   			 or (substring(per.phone, 2, 3) = '927' and cast(substring(per.phone, 5, 7) as numeric) between 0300000 and 0399999)
	   			 or (substring(per.phone, 2, 3) = '927' and cast(substring(per.phone, 5, 7) as numeric) between 0400000 and 0499999)
	   			 or (substring(per.phone, 2, 3) = '927' and cast(substring(per.phone, 5, 7) as numeric) between 2400000 and 2499999)
	   			 or (substring(per.phone, 2, 3) = '927' and cast(substring(per.phone, 5, 7) as numeric) between 4000000 and 4499999)
	   			 or (substring(per.phone, 2, 3) = '927' and cast(substring(per.phone, 5, 7) as numeric) between 4500000 and 4999999)
	   			 or (substring(per.phone, 2, 3) = '927' and cast(substring(per.phone, 5, 7) as numeric) between 6700000 and 6799999)
	   			 or (substring(per.phone, 2, 3) = '937' and cast(substring(per.phone, 5, 7) as numeric) between 2900000 and 2999999)
	   			 or (substring(per.phone, 2, 3) = '950' and cast(substring(per.phone, 5, 7) as numeric) between 3100000 and 3299999)
	   			 or (substring(per.phone, 2, 3) = '950' and cast(substring(per.phone, 5, 7) as numeric) between 6650000 and 6689999)
	   			 or (substring(per.phone, 2, 3) = '951' and cast(substring(per.phone, 5, 7) as numeric) between 0600000 and 0699999)
	   			 or (substring(per.phone, 2, 3) = '951' and cast(substring(per.phone, 5, 7) as numeric) between 8900000 and 8999999)
	   			 or (substring(per.phone, 2, 3) = '960' and cast(substring(per.phone, 5, 7) as numeric) between 0300000 and 0899999)
	   			 or (substring(per.phone, 2, 3) = '962' and cast(substring(per.phone, 5, 7) as numeric) between 5480000 and 5799999)
	   			 or (substring(per.phone, 2, 3) = '963' and cast(substring(per.phone, 5, 7) as numeric) between 1200000 and 1259999)
	   			 or (substring(per.phone, 2, 3) = '987' and cast(substring(per.phone, 5, 7) as numeric) between 2900000 and 2999999)
	   			) and length(phone) = 11
	   			)
	   		then 'Республика Татарстан'
	   		else 'Другой регион'
	   end region--,
	   --a.created + 3 * interval '1 hour'
from appointment a
left join patient p on a.patient_id = p.id 
left join person per on p.person_id = per.id 
left join promotion promo on a.promotion_id = promo.id
left join partner par on promo.partner_id = par.id
where a.created + 3 * interval '1 hour' < timezone('Europe/Moscow', now() - (extract(hour from now())) * interval '1 hour' - (extract(minute from now())) * interval '1 minute' - (extract(second from now())) * interval '1 second')
and a.good
and a.patient_id not in (
select patient_id
from patient_categories pcat
inner join reference rf
      on rf.id=pcat.reference_id
where rf.code='TEST'
    )
)
--select count(distinct(person_id)) 
--from temp
--where name = 'VEBMED'
--select count (distinct(phone)) from temp
select * from temp

"""

In [4]:
connection = UniversalConnection('../../.credentials/telemed/prom.cfg', DBType.Postgres)

2020-04-08 17:31:59 connect to postgres database using config file "../../.credentials/telemed/prom.cfg"
2020-04-08 17:31:59 creating ssh tunnel to 172.16.100.19 as root...
2020-04-08 17:32:10 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 36367
2020-04-08 17:32:10 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None


In [5]:
data = connection.query(sql)

2020-04-08 17:32:10 @telemed query:
                    with temp as (
				    select a.id appoint_id,
				    	   p.person_id,
				    	   per.phone,
				    	   --per.partner_id,
				    	   --par."name", 
				    	   case 
				            	when promo.description_tsvector = 'телемедицина_вэб' then 'VEBMED'
				            	else 'ПРОЧИЕ' 
				           end "name",
				    	   case
				    	   		when (((substring(per.phone, 2, 3) = '901' and cast(substring(per.phone, 5, 7) as numeric) between 9460000 and 9469999) 
				    	   			 or (substring(per.phone, 2, 3) = '902' and cast(substring(per.phone, 5, 7) as numeric) between 7110000 and 7119999)
				    	   			 or (substring(per.phone, 2, 3) = '902' and cast(substring(per.phone, 5, 7) as numeric) between 7150000 and 7159999)
				    	   			 or (substring(per.phone, 2, 3) = '902' and cast(substring(per.phone, 5, 7) as numeric) between 7180000 and 7199999)
				    	   			 or (substring(per.phone, 2, 3) = '903' and cast(substring(per.phone

In [6]:
connection.close()

2020-04-08 17:32:14 @telemed: closing postgres connection...
2020-04-08 17:32:14 closing ssh tunnel to 172.16.100.19...


In [7]:
df = pd.DataFrame(data)

---------

------

In [8]:
# тут все аналогично пациентам
df['phone'] = df['phone'].apply(valid_phone)

total_appointment = df.shape[0]

appointment_with_valid_phone = total_appointment - df[df['phone'] == 'Invalid phone'].shape[0]

appointment_web = df[(df['phone'] != 'Invalid phone') & 
                         (df['name'] == 'VEBMED')].shape[0]

appointment_tatarstan = df[(df['phone'] != 'Invalid phone') & 
                               (df['region'] == 'Республика Татарстан')].shape[0]

appointment_tatarstan_web = df[(df['phone'] != 'Invalid phone') & 
                                   (df['region'] == 'Республика Татарстан') & 
                                   (df['name'] == 'VEBMED')].shape[0]

In [9]:
total_patients = len(df.person_id.unique())

patients_with_valid_phone = len(df.phone.unique()) - 1 # нимаем телефон-инвалид

patients_web = len(df[df['name'] == 'VEBMED'].groupby('phone')['name'].count())

patient_tatarstan = len(df[df['region'] == 'Республика Татарстан'].groupby('phone')['name'].count())

patient_tatarstan_web = len(df[(df['region'] == 'Республика Татарстан') & (df['name'] == 'VEBMED')].groupby('phone')['name'].count())

In [10]:
final_report = {
          'Всего': [total_patients, total_appointment],
          'С телефоном': [patients_with_valid_phone, appointment_with_valid_phone],
          'По VEBMED': [patients_web, appointment_web],
          'Из Татарстана': [patient_tatarstan, appointment_tatarstan],
          'По VEBMED из Татарстана': [patient_tatarstan_web, appointment_tatarstan_web]   
}

In [11]:
report = pd.DataFrame.from_dict(final_report, orient='index')

In [12]:
report.columns = ['Пациентов', 'Консультаций']

In [13]:
report

,Пациентов,Консультаций
Всего,39559,92533
С телефоном,39287,92484
По VEBMED,2104,2707
Из Татарстана,723,1849
По VEBMED из Татарстана,99,115


In [14]:
report.to_csv('Report_Tatarstan.csv')

In [15]:
total_patients = len(df.person_id.unique())
total_patients

39559

In [16]:
patients_with_valid_phone = len(df.phone.unique()) - 1 # нимаем телефон-инвалид
patients_with_valid_phone

39287

In [17]:
patients_web = len(df[df['name'] == 'VEBMED'].groupby('phone')['name'].count())
patients_web

2104

In [18]:
patient_tatarstan = len(df[df['region'] == 'Республика Татарстан'].groupby('phone')['name'].count())
patient_tatarstan

723

In [19]:
patient_tatarstan_web = len(df[(df['region'] == 'Республика Татарстан') & 
                               (df['name'] == 'VEBMED')].groupby('phone')['name'].count())
patient_tatarstan_web

99

In [20]:
df[df['name'] == 'VEBMED'].groupby('phone')['name'].count()

phone
77755522244      2
79000195510      1
79000198486      1
79000427442      1
79000540935      3
                ..
79999783151      1
79999791334      4
79999792404      1
79999883963      1
Invalid phone    5
Name: name, Length: 2104, dtype: int64

In [21]:
df[df['region'] == 'Республика Татарстан'].groupby('phone')['name'].count()

phone
79027113858    1
79027114540    2
79027157202    1
79027199087    2
79030620182    2
              ..
79872957854    1
79872962479    2
79872976129    5
79872986204    3
79872987369    1
Name: name, Length: 723, dtype: int64

In [22]:
df[(df['region'] == 'Республика Татарстан') & (df['name'] == 'VEBMED')].groupby('phone')['name'].count()

phone
79027157202    1
79033056942    1
79033057556    1
79033067412    1
79033180744    1
              ..
79625591201    1
79625655621    1
79625770462    1
79872946064    1
79872957378    1
Name: name, Length: 99, dtype: int64